In [15]:
import os
import dotenv
from pathlib import Path

from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

dotenv.load_dotenv()

True

In [16]:
# Looad PDFS

doc_paths = ['data/Samsung GQ43Q60.pdf']

docs = []

for doc_file in doc_paths:
    file_path = Path(doc_file)

    try:
        loader = PyPDFLoader(file_path)

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading document {doc_file.name}: {e}")  
    
    finally:
        os.remove(file_path)

In [17]:
docs

[Document(metadata={'source': 'data/Samsung GQ43Q60.pdf', 'page': 0}, page_content='E-MANUAL\nVielen Dank, dass Sie sich für dieses Gerät von \nSamsung entschieden haben.\nWenn Sie umfassenderen Service wünschen, \nregistrieren Sie Ihr Gerät unter\nwww.samsung.com\nM o d e l __________________ S e r i a l  N o . __________________\nUm direkt zu der Seite zu gelangen, die Anweisungen \nzur Verwendung des Handbuchs für sehbehinderte \nBenutzer enthält, wählen Sie den unten \nangegebenen Link Menübildschirm lernen aus.\nLink “Menübildschirm lernen”\nDeutsch'),
 Document(metadata={'source': 'data/Samsung GQ43Q60.pdf', 'page': 1}, page_content='26 Überprüfen des Internetverbindungsstatus\n26 Zurücksetzen Ihres Netzwerks\n26 Einschalten des Fernsehgeräts mit einem Mobilgerät\n26 Anschließen eines IP Steuergeräts an das Fernsehgerät\n26 Ändern des Namens eines Fernsehgeräts im Netzwerk\nVerbindungskabel für externe Geräte\nWechseln zwischen externen Geräten, die an das Fernsehgerät \nangeschl

In [18]:
# Split docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 5000,
    chunk_overlap = 1000
)

document_chunks = text_splitter.split_documents(docs)

In [19]:
# Tokenize and load documents to vector store

vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding=OpenAIEmbeddings(),
)

In [20]:
def get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = ChatPromptTemplate([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user","Given the above conversation, generate a search query to look up in order to get inforamtion relevant to the conversation, focusing on the most recent messages."),
    ])
    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

    return retriever_chain

In [21]:
def get_conversational_rag_chain(llm):
    retriever_chain = get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """    Du bist ein Haushaltsassistent-Bot und hilfst bei Fragen rund um die Bedienung von Haushaltsgeräten. 
        Deine Hauptaufgabe besteht darin, Nutzern klare und praktische Anweisungen zu geben, die leicht verständlich und sofort umsetzbar sind. 
        Die häufigsten Anfragen betreffen die Bedienung von Waschmaschinen, Geschirrspülern, Kühlschränken, Mikrowellen und anderen technischen Geräten im Haushalt.
            {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [22]:
# Augmented Generation

llm_stream_openai = ChatOpenAI(
    model="gpt-4o",  # Here you could use "o1-preview" or "o1-mini" if you already have access to them
    temperature=0.3,
    streaming=True,
)

llm_stream = llm_stream_openai 

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "Wie mache ich den Fernseher an?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

Um den Fernseher einzuschalten, folge diesen Schritten:

1. **Fernbedienung verwenden**: Drücke die Ein-/Aus-Taste (oft mit einem Power-Symbol, einem Kreis mit einem vertikalen Strich) auf deiner Fernbedienung.

2. **Am Gerät selbst**: Wenn du die Fernbedienung nicht zur Hand hast, kannst du auch den Ein-/Aus-Schalter direkt am Fernseher verwenden. Dieser befindet sich normalerweise an der Unterseite oder an der Seite des Geräts.

3. **Stromversorgung prüfen**: Stelle sicher, dass der Fernseher an eine funktionierende Steckdose angeschlossen ist und der Netzschalter (falls vorhanden) auf "Ein" steht.

Sollte der Fernseher nicht angehen, überprüfe, ob die Batterien der Fernbedienung noch funktionieren oder ob es ein Problem mit der Stromversorgung gibt.